In [822]:
import numpy as np
from numpy.linalg import inv

Define dictionary `layer` assigning a layer label with fixed, potentially complex elastic constants and density. The elastic constants correspond to an VTI medium and are to be given in order ($c_{11}, c_{13}, c_{33}, c_{44}, \rho$)

In [823]:
layers={
    "layer 1":(22.56, 12.38, 17.35, 3.15, 2.38),
    "layer 2":(26.73, 12.51, 26.73, 7.11, 2.22),
    "layer 3":(11.25, 6.066, 11.25, 2.592, 1.8),
    "layer 4":(11.25 + 1j, 6.066, 26.25, 2.592, 1.8),
    "layer 5":(11.25 + 1j, 2.066, 19.25, 2.592, 1.8),
    "layer 6":(11.25 + 1j, 6.066, 23.25, 2.592, 1.8)
};

In [3]:
layers["layer 1"]

(22.56, 12.38, 17.35, 3.15, 2.38)

In [824]:
def tlQLmatrices(layer, p):
    # based on Ursin and Stovas, Geophysics (2002)
    [c11, c13, c33, c44, d]=layers[layer];
    a0 = sqrt(c33/d);
    b0 = sqrt(c44/d);
    s0 = 1-c44/c33;
    dD = (c13 - c33 + 2*c44)/c33;
    eta = (c11*c33 - (c13 + 2*c44)**2)/(2*c33**2);
    R1 = 2*(1 - p**2*b0**2)*(dD + 2*p**2*a0**2*eta)**2;
    R2 = s0 + 2*p**2*b0**2*dD - 2*p**2*a0**2*(1 - 2*p**2*b0**2)*eta;
    R = R1/(R2 + sqrt(R2**2 + 2*p**2*b0**2*R1));
    Sa = 2*dD + 2*p**2*a0**2*eta + R
    Sb = 2*(1 - p**2*b0**2)*a0**2/b0**2*eta - R
    q1 = 1/a0**2 - p**2 - p**2*Sa;
    q2 = 1/b0**2 - p**2 - p**2*Sb;
    qa = sqrt(q1.real + 1j*abs(q1.imag));
    qb = sqrt(q2.real + 1j*abs(q2.imag));
    d2 = sqrt((s0 + dD)/(s0 + Sa));
    d3 = 2*b0**2*(s0 + 
     1/2*(Sa + dD))/(s0 + dD);
    d4 = sqrt((s0 - p**2*b0**2*(s0 + Sb))/((1 - p**2*b0**2*(1 + Sb))*(s0 + dD)));
    d5 = (s0 - 2*p**2*b0**2*(s0 + 1/2*(Sb + dD)))/(s0 + dD);
    d1 = 1/sqrt(p**2*d3 + d5);
    L1 = d1*np.array([[d2*sqrt(qa/d), 1/d4*p/sqrt(d*qb)],
                      [d3*d2*p*sqrt(d*qa), -d5/d4*sqrt(d/qb)]
                     ], dtype = np.complex_);
    L2 = d1*np.array([[d5/d2*sqrt(d/qa), d3*d4*p*sqrt(d*qb)],
                      [1/d2*p/sqrt(d*qa), -d4*sqrt(qb/d)]
                     ], dtype = np.complex_);
    return(np.array([diagflat([qa,qb]),L1,L2]))


In [825]:
tlQLmatrices("layer 1", .1)

array([[[ 0.35418607+0.j,  0.        +0.j],
        [ 0.        +0.j,  0.85905696+0.j]],

       [[ 0.38519236+0.j,  0.07693826+0.j],
        [ 0.25570415+0.j, -1.60740115+0.j]],

       [[ 2.516156  +0.j,  0.40026818+0.j],
        [ 0.12043581+0.j, -0.6029634 +0.j]]])

In [168]:
def tlLayerReflectionTransmission(layer1, layer2, p):
    # based on Ursin and Stovas, Geophysics (2002)
    # no need to use slownesses. Get the L matrices only
    # and use equations B-1, B-2 to calculate T, R
    [L1t, L2t] = tlQLmatrices(layer1, p)[1:5];
    [L1b, L2b] = tlQLmatrices(layer2, p)[1:5];
    
    # C and D as defined in eq. B-2
    C = np.dot(L2b.T, L1t);
    D = np.dot(L1b.T, L2t);
    
    # Reflection and transmission matrices of a single interface
    trans = 2*inv(C + D).T
    refl = np.dot((C-D).T,trans/2)
    
    return(np.array([trans,refl]))

In [169]:
tlLayerReflectionTransmission("layer 1","layer 3",.2)

array([[[ 0.97989791+0.j,  0.01004981+0.j],
        [-0.01690589+0.j,  0.99439359+0.j]],

       [[-0.18750511+0.j, -0.06600089+0.j],
        [-0.06600089+0.j,  0.08200169+0.j]]])

In [819]:
 def tlReflectivity(layers, thicknesses, p, w):
    # based on Ursin and Stovas, Geophysics (2002)
    # Recursion is eq. 20
    n_of_layers = len(layers)-1;
    
    # QL matrices for layer stack
    QL = [tlQLmatrices(i,p) for i in layers];
    
    # Pairwise reflection transmission for layer stack
    RT = [tlLayerReflectionTransmission(layers[i],layers[i+1], p) for i in range(n_of_layers)]
    
    # Phase propagator for layer stack
    phase = [diagflat(exp(1j*w*i[0]*diag(i[1]))) for i in zip(thicknesses,[item[0] for item in QL])]
    
    # Recursive calculation of reflection response
    def rec(n):
        if n == n_of_layers-1:
            return(zeros(shape=(2,2), dtype=np.complex_))
        else:
            inv_mat = inv(identity(2, dtype=np.complex_) + dot(RT[n+1][1],rec(n+1)))
            inv_t = dot(transpose(RT[n+1][0]) , dot(rec(n+1) , dot(inv_mat , RT[n+1][0])))
            return(
                dot(
                    dot(
                        phase[n + 1],
                       RT[n + 1][1] + inv_t
                    )
                        ,phase[n + 1]
                   )
            )
    return(rec(-1))

In [821]:
timeit.timeit('[tlReflectivity(["layer 1","layer 2", "layer 5", "layer 2", "layer 5",  "layer 3","layer 6"],[3,2,5,4,3,4,12],0.2,i) for i in range(100)]', 'from __main__ import tlReflectivity, tlQLmatrices, layers', number=100)

63.16480337100802

In [820]:
tlReflectivity(["layer 1", "layer 3", "layer 4", "layer 6","layer 5", "layer 6","layer 2"],[0,.4,.1,.2,.3,.2,0],1,1)

array([[ 0.20813128+0.00953675j, -0.00374497-0.01095699j],
       [-0.00374497-0.01095699j,  0.34048943+0.00586548j]])